In [1]:
import pandas as pd
import json
import numpy as np
import seaborn as sn
import pickle

from matplotlib import pyplot as plt
from io import StringIO
%matplotlib inline

In [2]:
import os
import ssl
import requests

from tqdm import tqdm
from PIL import Image
from io import BytesIO
from urllib import request

In [3]:
df_listings_details = pd.read_csv('../data/listings_detailed.csv')
df_listings = pd.read_csv('../data/listings.csv')

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('future.no_silent_downcasting', True)

In [5]:
selected_columns = ['id','room_type', 'minimum_nights', 'neighbourhood',
   'availability_eoy', 'availability_365', 'picture_url',
    'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'host_identity_verified',
    'accommodates', 'bathrooms', 'bedrooms', 'beds',
    'estimated_occupancy_l365d', 'estimated_revenue_l365d',
    'number_of_reviews', 'number_of_reviews_l30d', 'reviews_per_month', 
    'review_scores_rating', 'review_scores_value', 
    'instant_bookable', 'calculated_host_listings_count', 'price']


In [6]:
def normalize_tf_cols(df, column):
    df[column] = df[column].replace({'t': 1, 'f': 0}).astype(bool)
    return df

In [7]:
def fix_encoding(df_cleaned):
    encoding_map = {}
    for val in df_cleaned['neighbourhood'].unique():
        try:
            clean_val = val.encode("latin1").decode("utf-8", errors="ignore")
            encoding_map[val] = clean_val
        except (UnicodeEncodeError, AttributeError):
            encoding_map[val] = val
    df_cleaned['neighbourhood'] = df_cleaned['neighbourhood'].map(encoding_map)
    return df_cleaned

In [8]:
def data_cleanup(df_1, df_2):
    df_merged_listings = pd.concat([df_listings, df_listings_details], axis=1)
    df_merged_listings = df_merged_listings.loc[:, ~df_merged_listings.columns.duplicated()]
    df_cleaned = df_merged_listings[selected_columns].dropna() 
    df_cleaned = df_cleaned[df_cleaned['availability_eoy']> 0]
    df_cleaned = df_cleaned[df_cleaned['availability_365']> 0]
    df_cleaned = df_cleaned[df_cleaned['estimated_occupancy_l365d']> 0]
    df_cleaned['host_response_rate'] = df_cleaned['host_response_rate'].str.replace('%', '', regex=False).astype(float)
    df_cleaned['host_acceptance_rate'] = df_cleaned['host_acceptance_rate'].str.replace('%', '', regex=False).astype(float)
    df_cleaned = normalize_tf_cols(df_cleaned, 'instant_bookable')
    df_cleaned = normalize_tf_cols(df_cleaned, 'host_identity_verified')
    df_cleaned = normalize_tf_cols(df_cleaned, 'host_is_superhost')
    df_cleaned = fix_encoding(df_cleaned)
    df_cleaned.columns = df_cleaned.columns.str.replace('/','_')
    df_cleaned.columns = df_cleaned.columns.str.lower()
    df_cleaned.columns = df_cleaned.columns.str.replace(' ','_')
    return df_cleaned

In [9]:
df_cleaned = data_cleanup(df_listings, df_listings_details)

In [10]:
df_cleaned = df_cleaned.reset_index(drop=True)
df_cleaned.head(5)

,id,room_type,minimum_nights,neighbourhood,availability_eoy,availability_365,picture_url,host_response_rate,host_acceptance_rate,host_is_superhost,host_identity_verified,accommodates,bathrooms,bedrooms,beds,estimated_occupancy_l365d,estimated_revenue_l365d,number_of_reviews,number_of_reviews_l30d,reviews_per_month,review_scores_rating,review_scores_value,instant_bookable,calculated_host_listings_count,price
0,51287,Entire home/apt,30,Leopoldstadt,46,207,https://a0.muscache.com/pictures/25163038/1c4e...,100.0,100.0,True,True,2,1.0,0.0,2.0,180,12780.0,383,0,2.15,4.67,4.59,False,2,71.0
1,169672,Private room,2,Wieden,8,255,https://a0.muscache.com/pictures/c1a1e093-66da...,100.0,69.0,True,True,2,1.0,1.0,1.0,120,6960.0,445,3,2.59,4.77,4.77,False,1,58.0
2,169752,Entire home/apt,1,Margareten,26,251,https://a0.muscache.com/pictures/97156e49-9dbd...,100.0,99.0,True,True,3,1.0,0.0,1.0,54,3132.0,112,2,0.89,4.85,4.75,True,11,58.0
3,171835,Entire home/apt,3,Wieden,4,4,https://a0.muscache.com/pictures/2077532/45431...,100.0,67.0,True,True,3,1.0,2.0,2.0,72,5832.0,195,1,1.15,4.68,4.73,False,1,81.0
4,172530,Entire home/apt,1,Margareten,9,171,https://a0.muscache.com/pictures/1134938/f9bec...,100.0,99.0,True,True,7,1.5,3.0,4.0,30,4140.0,243,0,1.42,4.78,4.63,True,11,138.0


In [11]:
def identify_premium_properties(df, threshold=0.5):
    neighborhood_premium_stats = df_cleaned.groupby('neighbourhood').agg({
        'id':'count',
        'price':  lambda x: x.quantile(threshold),
        'review_scores_value': lambda x: x.quantile(threshold)
    })
    neighborhood_premium_stats = neighborhood_premium_stats.rename(
    columns={
        'price': 'price_q_threshold',
        'review_scores_value': 'rating_q_threshold'
    })
    neighborhood_premium_stats = neighborhood_premium_stats.reset_index()
    df_premium = df_cleaned.merge(
        neighborhood_premium_stats[['neighbourhood', 'price_q_threshold', 'rating_q_threshold']],
        on='neighbourhood',
        how='left'
    )
    df_premium['is_premium'] = (
            (df_premium['price'] >= df_premium['price_q_threshold']) &
            (df_premium['review_scores_value'] >= df_premium['rating_q_threshold'])
        )
    df_premium['is_premium'] = df_premium['is_premium'].astype(int)
    return df_premium

In [12]:
df_premium = identify_premium_properties(df_cleaned, 0.5)

In [13]:
len(df_cleaned),len(df_premium),len(df_premium[df_premium['is_premium'] == 1]), len(df_premium[df_premium['is_premium'] == 0])

(6794, 6794, 1907, 4887)

In [14]:
df_premium[['id', 'picture_url']].head(2)

,id,picture_url
0,51287,https://a0.muscache.com/pictures/25163038/1c4e...
1,169672,https://a0.muscache.com/pictures/c1a1e093-66da...


In [15]:
def download_image(url, save_path):
    try:
        r = requests.get(url, timeout=10)
        if r.status_code == 200:
            with open(save_path, "wb") as f:
                f.write(r.content)
            print("Image found")
            return True
    except:
        print("Image Not found exception")
        pass
    print("Image Not found")
    return False

In [16]:
def download_image_from_url(url):
    context = ssl._create_unverified_context()
    try:
        with request.urlopen(url, context=context) as resp:
            buffer = resp.read()
        stream = BytesIO(buffer)
        img = Image.open(stream)
        return img
    except:
        print("Image Not found exception")
        return None
    print("Image Not found")
    return None

In [17]:
def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [41]:
sample_df = df_premium.sample(100, random_state=42)

for _, row in tqdm(sample_df.iterrows(), total=len(sample_df)):
    url = row['picture_url']
    listing_id = row['id']
    
    if pd.isna(url):
        continue
    
    if row['is_premium'] == 1:
        folder = "../images/premium"
    else:
        folder = "../images/non_premium"
    
    save_path = f"{folder}/{listing_id}.jpg"
    print(save_path)
    img = download_image_from_url(url)

    if img is None:
        print(img)
        continue;
        
    resized_img =  prepare_image(img, (300,400))
    resized_img.save(save_path) 

  0%|                                                                                                     | 0/100 [00:00<?, ?it/s]

../images/non_premium/964206565905955937.jpg


  1%|▉                                                                                            | 1/100 [00:00<00:25,  3.95it/s]

../images/non_premium/1102298634083944389.jpg
../images/non_premium/1412226864397489130.jpg


  3%|██▊                                                                                          | 3/100 [00:01<00:35,  2.73it/s]

...

 96%|████████████████████████████████████████████████████████████████████████████████████████▎   | 96/100 [00:20<00:00,  5.76it/s]

../images/non_premium/1389831309311365517.jpg


 99%|███████████████████████████████████████████████████████████████████████████████████████████ | 99/100 [00:20<00:00,  6.36it/s]

../images/non_premium/1424555510816756744.jpg
../images/non_premium/44027879.jpg
../images/non_premium/34876910.jpg


100%|███████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:20<00:00,  4.77it/s]
